# Processing multiple interferograms with ISCE

In [4]:
# Import required packages
import logging
log = logging.getLogger()
log.setLevel(logging.WARN)
import os
import getpass
import asf_search as asf
import isce
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
2022-09-29 17:23:28,492 - matplotlib - DEBUG - matplotlib data path: /home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/matplotlib/mpl-data
2022-09-29 17:23:28,499 - matplotlib - DEBUG - CONFIGDIR=/home/jovyan/.config/matplotlib
2022-09-29 17:23:28,502 - matplotlib - DEBUG - interactive is False
2022-09-29 17:23:28,503 - matplotlib - DEBUG - platform is linux
2022-09-29 17:23:28,503 - matplotlib - DEBUG - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakr

In [ ]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)
print(os.environ['PATH'])

In [ ]:
!which topsApp.py

In [1]:
scene_list = ['S1B_IW_SLC__1SDV_20200717T005351_20200717T005418_022504_02AB6A_333A',
              'S1B_IW_SLC__1SDV_20200717T005351_20200717T005418_022504_02AB6A_333A',
              'S1B_IW_SLC__1SDV_20200722T010143_20200722T010210_022577_02AD97_0AF9',
              'S1B_IW_SLC__1SDV_20200722T010208_20200722T010235_022577_02AD97_AE1D',
              'S1B_IW_SLC__1SDV_20200729T005352_20200729T005419_022679_02B0C0_8427',
              'S1B_IW_SLC__1SDV_20200803T010144_20200803T010211_022752_02B2E3_2F86',
              'S1B_IW_SLC__1SDV_20200803T010209_20200803T010236_022752_02B2E3_C178',
              'S1B_IW_SLC__1SDV_20200810T005352_20200810T005419_022854_02B621_D8F8',
              'S1B_IW_SLC__1SDV_20200815T010145_20200815T010212_022927_02B84B_AD30',
              'S1B_IW_SLC__1SDV_20200815T010210_20200815T010237_022927_02B84B_FCB6',
              'S1B_IW_SLC__1SDV_20200822T005353_20200822T005420_023029_02BB99_67C5',
              'S1B_IW_SLC__1SDV_20200827T010146_20200827T010213_023102_02BDCC_5F7D',
              'S1B_IW_SLC__1SDV_20200827T010211_20200827T010238_023102_02BDCC_BB77',
              'S1B_IW_SLC__1SDV_20200903T005354_20200903T005421_023204_02C10B_ABD8',
              'S1B_IW_SLC__1SDV_20200908T010146_20200908T010213_023277_02C345_9DE5',
              'S1B_IW_SLC__1SDV_20200908T010211_20200908T010238_023277_02C345_0255'
             ]

In [5]:
def select_pairs(scene_list, max_temp_bline):
    scene_dates = {}
    for scene in scene_list:
        date = scene[17:25]
        scene_dates[date] = scene
        
    pair_dict = {}
    pair_scenes = []
    for date1 in scene_dates:
        for date2 in scene_dates:
            if datetime.strptime(date2, '%Y%m%d')-datetime.strptime(date1, '%Y%m%d') < timedelta(days=max_temp_bline) and not date1 >= date2 :
                pair_dict[f'{date1}-{date2}'] = [scene_dates[date1], scene_dates[date2]]
                pair_scenes.append(scene_dates[date1])
                pair_scenes.append(scene_dates[date1])
    pair_scenes = [*set(pair_scenes)]
    
    return pair_dict, pair_scenes

In [6]:
pair_dict, pair_scenes = select_pairs(scene_list, 12)

In [ ]:
proc_path = '/home/jovyan/rmnp_landslide/proc'

for pair in pair_dict:
    pair_path = f'{proc_path}/{pair}'
    if not os.path.exists(pair_dir}:
                          os.makedirs(pair_path)

## Download all SLCs

In [ ]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

In [ ]:
# Change to SLC directory
%%time

os.chdir(f'{proc_path}/slc')

logging.getLogger("urllib3").setLevel(logging.WARNING)

results = asf.granule_search(pair_scenes)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path=f'{proc_path}/slc', processes=2, session=session)

## Download orbital files

In [ ]:
os.chdir(f'{proc_path}/orbital')

In [ ]:
%%bash

wget -nc https://raw.githubusercontent.com/isce-framework/isce2/main/contrib/stack/topsStack/fetchOrbit.py
chmod +x fetchOrbit.py

In [ ]:
# grab orbital files with fetchOrbit.py
for scene in pair_scenes:
    os.system(f'./fetchOrbit.py -i {scene}')

## Write input files

In [ ]:
for pair in pair_dict:
    os.chdir(f'{proc_path}/{pair}')
    
    reference = pair_dict[pair][0]
    secondary = pair_dict[pair][1]
    
    with open('topsApp.xml', 'w') as f:
        f.write(f"""<?xml version="1.0" encoding="UTF-8"?>
    <topsApp>
      <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>
        <component name="reference">
            <property name="orbit directory">{proc_path}/orbital</property>
            <property name="output directory">reference</property>
            <property name="safe">{proc_path}/slc/{reference}.zip</property>
        </component>
        <component name="secondary">
            <property name="orbit directory">{proc_path}/orbital</property>
            <property name="output directory">secondary</property>
            <property name="safe">{proc_path}/slc/{secondary}.zip</property>
        </component>
        <property name="demfilename">{proc_path}/usgs_10m.dem.wgs84</property>
        <property name="useGPU">True</property>
        <property name="range looks">6</property>
        <property name="azimuth looks">1</property>
        <property name="region of interest">[40.295, 40.310, -105.69, -105.67]</property>
        <property name="do esd">False</property>
        <property name="do ionospherecorrection">False</property>
        <property name="do unwrap">True</property>
        <property name="unwrapper name">snaphu_mcf</property>
        <property name="do denseoffsets">False</property>
        <property name="geocode demfilename">{proc_path}/usgs_10m.dem.wgs84</property>
        <property name="geocode list">['merged/topophase.cor', 'merged/filt_topophase.unw', 'merged/filt_topophase.flat']</property>
      </component>
    </topsApp>""")

## TopsApp processing

In [ ]:
%%time

for pair in pair_dict:
    os.chdir(f'{proc_path}/{pair}')
    
    !topsApp.py --start=preprocess --end=geocode
    